Linear algebra
--------------

Silent substitution problems can also be solved with linear algebra. Suppose 

Algebraic solutions to silent substitution problems are computationally inexpensive and produce zero error in linear systems. Assume we have a 4-primary stimulation device with 8-bit resolution (256 steps). As a background spectrum we use each LED at half maximum

Linear algebra can produce solutions that match photoreceptor activations . Generally speaking, they are fast and computationally inexpensive. Howevr, there is only a one solution 

- there exists only one algebraic solution. 

To calculate photoreceptor activation for a spectrum we can weight

For a given spectrum, we can calculate photoreceptor activation by 

We can calculate photoreceptor activation by taking the sum of the 

The choice of background will vary depending on the research goal. The stimulus problem will . The amount of contrast available is determined by the gamut of the stimlation device. 

The background spectrum mai
Overview of various stimulus problems, with examples drawn from publications

Contrast is the difference in photoreceptor activation between the modulation and background spectra

Weber (unipolar) contrast:

$$C_{W}=\frac{max-min}{max}$$

Michelson (bipolar) contrast:

$C_{M}=\frac{max-min}{max+min}$

Contrast can be specified as either fractions or percentages

Pulses of contrast against a background, or temporal modulations

negative contrast

Which spectral sensetivities are assumed?

In [1]:
import pandas as pd

import pandas as pd

from pysilsub.problem import SilentSubstitutionProblem as SSP

# Load the calibration data
spds = pd.read_csv(
    '../../data/BCGAR_5_Primary_8_bit_linear.csv', 
    index_col=['Primary','Setting'])
spds.columns = pd.Int64Index(spds.columns.astype(int))
spds.columns.name = 'Wavelength'

# List of colors for the primaries
colors = ['blue', 'cyan', 'green', 'orange', 'red'] 

ssp = SSP(
    resolutions=[255]*5,  # Five 8-bit primaries
    colors=colors,  # Colors of the LEDs
    spds=spds,  # The calibration data
    spd_binwidth=1,  # SPD wavelength binwidth
    ignore=['R'],  # Ignore rods
    silence=['S', 'M', 'L'],  # Silence S-, M-, and L-cones
    isolate=['I'],  # Isolate melanopsin
    target_contrast=2.5,  # Aim for 250% contrast 
    name='BCGAR (8-bit, linear)'  # Description of device
) 

SSProblem that encapsulates the issue and prepares for solutions with linear algebra and numerical optimisation.

Solving with linear algebra
---------------------------



Silent substitution problems can also be solved with linear algebra. Continuing with our 5-primary device, suppose that we now want to isolate S-cones. Borrowing the notation from [Cao et al. (2015)](https://jov.arvojournals.org/article.aspx?articleid=2213232), let's say that the spectral power distributions of the five primaries at maximum are $\tilde{P_{0}}(\lambda), \tilde{P_{1}}(\lambda), \tilde{P_{2}}(\lambda), \tilde{P_{3}}(\lambda), \tilde{P_{4}}(\lambda)$, with $\alpha = [p_{0}, p_{1}, p_{2}, p_{3}, p_{4}]$ representing the proportion of the maximum for each LED, the photoreceptor excitations $\beta = [S M L R I]$ can be calculated algebraicly with $\alpha = \beta A$, where $A$ is a matrix representing the *a*-opic irradiances for each primary at its maximum setting:

$A=\begin{bmatrix} S_{0} & S_{1} & S_{2} & S_{3} & S_{4}\\ M_{0} & M_{1} & M_{2} & M_{3} & M_{4}\\ L_{0} & L_{1} & L_{2} & L_{3} & L_{4}\\  R_{0} & R_{1} & R_{2} & R_{3} & R_{4}\\ I_{0} & I_{1} & I_{2} & I_{3} & I_{4}\end{bmatrix}$

Now, scaling coefficient is $\alpha = \beta A^{-1}$

In [5]:
bg = [.5, .5, .5, .5, .5]
mod = [.6, .5, .5, .5, .5]

In [6]:
Pmax = [1., 1., 1., 1., 1.]
Pmax_spds = ssp.predict_multiprimary_spd(Pmax, nosum=True)
Pmax_spds

Primary,0,1,2,3,4
Wavelength,,,,,
380,0.002188,0.001705,0.002222,0.003669,0.003459
381,0.002260,0.001859,0.002372,0.004118,0.004063
382,0.001974,0.001670,0.002387,0.003874,0.003610
383,0.002174,0.001650,0.002452,0.003874,0.004071
384,0.001751,0.001484,0.001999,0.003333,0.003099
...,...,...,...,...,...
776,0.001244,0.001062,0.001414,0.004679,0.002699
777,0.001150,0.001045,0.001381,0.004592,0.002592
778,0.001028,0.000793,0.001120,0.004275,0.002177


Next, we'll be needing the spectral sensitivities for the photoreceptors.

In [7]:
from pysilsub.CIE import get_CIES026

sss = get_CIES026()
sss

Photoreceptor,S,M,L,R,I
Wavelength,,,,,
380,0.0,0.000000,0.000000,5.890000e-04,9.181600e-04
381,0.0,0.000000,0.000000,6.650000e-04,1.045600e-03
382,0.0,0.000000,0.000000,7.520000e-04,1.178600e-03
383,0.0,0.000000,0.000000,8.540000e-04,1.322800e-03
384,0.0,0.000000,0.000000,9.720000e-04,1.483800e-03
...,...,...,...,...,...
776,0.0,0.000002,0.000024,1.730000e-07,2.550000e-08
777,0.0,0.000002,0.000023,1.640000e-07,2.420000e-08
778,0.0,0.000002,0.000021,1.550000e-07,2.290000e-08


The dot product of these matrices, which we will call the primary-to-receptor matrix, contains the *a*-opic irradiances for the primary components of the background spectrum.

In [9]:
A = sss.T.dot(Pmax_spds).T
A

Photoreceptor,S,M,L,R,I
Primary,,,,,
0,7.132159,1.250263,0.868063,3.447991,4.173141
1,4.888283,2.701134,1.740474,5.907189,7.014032
2,0.421072,7.118705,5.807598,7.249536,5.661086
3,0.189787,9.329613,17.305348,2.566504,1.014661
4,0.113690,0.312333,1.699980,0.152688,0.138191


The inverse of the primary-to-receptor matrix is ultimately what will help us find a solution.

In [13]:
import numpy as np

A1 = pd.DataFrame(
    np.linalg.inv(A.values),
    A.columns, 
    A.index)
A1.T

Photoreceptor,S,M,L,R,I
Primary,,,,,
0,0.237824,-0.079977,-0.000905,0.498480,-0.554540
1,-0.140289,0.170675,-0.018809,-0.830695,0.878890
2,-0.001217,-0.143764,0.007694,0.673637,-0.513031
3,0.013338,0.252159,-0.030002,-0.429614,0.262860
4,-0.109431,-2.209677,0.887082,2.667970,-1.539854


In [51]:
bp = A.T.dot([.5,.5,.5,.5,.5])
bp

Photoreceptor
S     6.372495
M    10.356024
L    13.710732
R     9.661954
I     9.000556
dtype: float64

In [55]:
mod = A.T.dot([.5,.5,.5,.5,.6])
mod

Photoreceptor
S     6.383864
M    10.387257
L    13.880730
R     9.677223
I     9.014375
dtype: float64

In [56]:
a = A1.T.dot(mod)
a

Primary
0    0.5
1    0.5
2    0.5
3    0.5
4    0.6
dtype: float64

In [41]:
A1.T.dot(bg)

Primary
0    0.050441
1    0.029886
2    0.011660
3    0.034371
4   -0.151955
dtype: float64

In [44]:
ssp.plot_ss_result((bg+A1.dot([0,0,0,0,0])).to_list() + (bg+A1.dot([0,0,0,0,.5])).to_list())

ValueError: Requested setting 467 exceeds resolution of device primary 3

Maguire, J., Parry, N. R. A., Kremers, J., Kommanapalli, D., Murray, I. J., & McKeefry, D. J. (2016). Rod electroretinograms elicited by silent substitution stimuli from the light-adapted human eye. Translational Vision Science and Technology, 5(4). https://doi.org/10.1167/tvst.5.4.13

Advantages of the algebraic approach:

- fast
- mathematically exact
- zero error in a linear system

Disadvantages:

- does not give max contrast available
- background options are limited
- generally inflexible

More info coming soon.